In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [2]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'
        
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        self.X = np.mat(X_train)
        self.y = np.mat(y_train).T
        
        if self.intercept:
            ones = np.ones((X_train.shape[0], 1))
            self.X = np.hstack((ones, self.X))
        
        if self.init_weight is None:
            self.init_weight = np.random.uniform(-1, 1, size=(self.X.shape[1], 1))
        self.coef = self.init_weight #### Please change this after you get the example right.

        self.gradient_descent()
        return
        
    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        predictions = self.X * self.coef
        error = predictions - self.y
        grad_coef = self.X.T * error / self.X.shape[0]
        return grad_coef
        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """
        
        self.loss = []
        
        for i in range(self.num_iter):
            grad = self.gradient()
            temp_coef = self.coef - self.alpha * grad

            previous_y_hat = self.X * self.coef
            pre_error = np.sum(np.square(self.y - previous_y_hat))

            current_y_hat = self.X * temp_coef
            current_error = np.sum(np.square(self.y - current_y_hat))

            if current_error <= pre_error:
                self.coef = temp_coef
                self.alpha *= 1.3
            else:
                self.alpha *= 0.9

            if (abs(pre_error - current_error) < self.early_stop) or (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                return self

            # self.loss.append(current_error)
            
            # if i % 10000 == 0:
            #     print('Iteration: ' +  str(i))
            #     print('Coef: '+ str(self.coef))
            #     print('Loss: ' + str(current_error))            
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        x = np.mat(x)
        result = x * self.coef
        return result[0, 0]
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
            TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        ret = []
        X = np.mat(X)
        if self.intercept:
            ones = np.ones((X.shape[0], 1))
            X = np.hstack((ones, X))
        for x in X:
            ret.append(self.ind_predict(x))
        return ret
        
        

In [3]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [4]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [5]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [6]:
clf.coef

matrix([[19.93498082],
        [30.00009741]])

In [7]:
np.array(clf.predict(X))

array([   49.93507823,   199.9355653 ,   349.93605236,   499.93653942,
         649.93702649,   799.93751355,   949.93800061,  1099.93848768,
        1249.93897474,  1399.9394618 ,  1549.93994887,  1699.94043593,
        1849.94092299,  1999.94141006,  2149.94189712,  2299.94238418,
        2449.94287125,  2599.94335831,  2749.94384537,  2899.94433244,
        3049.9448195 ,  3199.94530656,  3349.94579363,  3499.94628069,
        3649.94676775,  3799.94725482,  3949.94774188,  4099.94822894,
        4249.94871601,  4399.94920307,  4549.94969013,  4699.9501772 ,
        4849.95066426,  4999.95115132,  5149.95163839,  5299.95212545,
        5449.95261251,  5599.95309958,  5749.95358664,  5899.9540737 ,
        6049.95456077,  6199.95504783,  6349.95553489,  6499.95602195,
        6649.95650902,  6799.95699608,  6949.95748314,  7099.95797021,
        7249.95845727,  7399.95894433,  7549.9594314 ,  7699.95991846,
        7849.96040552,  7999.96089259,  8149.96137965,  8299.96186671,
      

In [8]:
X = min_max_normaliz(X)
y = min_max_normaliz(y)


#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [9]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [10]:
clf.coef

matrix([[6.96883989e-25],
        [1.00000000e+00]])

In [11]:
np.array(clf.predict(X))

array([6.96883989e-25, 5.02512563e-03, 1.00502513e-02, 1.50753769e-02,
       2.01005025e-02, 2.51256281e-02, 3.01507538e-02, 3.51758794e-02,
       4.02010050e-02, 4.52261307e-02, 5.02512563e-02, 5.52763819e-02,
       6.03015075e-02, 6.53266332e-02, 7.03517588e-02, 7.53768844e-02,
       8.04020101e-02, 8.54271357e-02, 9.04522613e-02, 9.54773869e-02,
       1.00502513e-01, 1.05527638e-01, 1.10552764e-01, 1.15577889e-01,
       1.20603015e-01, 1.25628141e-01, 1.30653266e-01, 1.35678392e-01,
       1.40703518e-01, 1.45728643e-01, 1.50753769e-01, 1.55778894e-01,
       1.60804020e-01, 1.65829146e-01, 1.70854271e-01, 1.75879397e-01,
       1.80904523e-01, 1.85929648e-01, 1.90954774e-01, 1.95979899e-01,
       2.01005025e-01, 2.06030151e-01, 2.11055276e-01, 2.16080402e-01,
       2.21105528e-01, 2.26130653e-01, 2.31155779e-01, 2.36180905e-01,
       2.41206030e-01, 2.46231156e-01, 2.51256281e-01, 2.56281407e-01,
       2.61306533e-01, 2.66331658e-01, 2.71356784e-01, 2.76381910e-01,
      

In [12]:
X

array([[0.        ],
       [0.00502513],
       [0.01005025],
       [0.01507538],
       [0.0201005 ],
       [0.02512563],
       [0.03015075],
       [0.03517588],
       [0.04020101],
       [0.04522613],
       [0.05025126],
       [0.05527638],
       [0.06030151],
       [0.06532663],
       [0.07035176],
       [0.07537688],
       [0.08040201],
       [0.08542714],
       [0.09045226],
       [0.09547739],
       [0.10050251],
       [0.10552764],
       [0.11055276],
       [0.11557789],
       [0.12060302],
       [0.12562814],
       [0.13065327],
       [0.13567839],
       [0.14070352],
       [0.14572864],
       [0.15075377],
       [0.15577889],
       [0.16080402],
       [0.16582915],
       [0.17085427],
       [0.1758794 ],
       [0.18090452],
       [0.18592965],
       [0.19095477],
       [0.1959799 ],
       [0.20100503],
       [0.20603015],
       [0.21105528],
       [0.2160804 ],
       [0.22110553],
       [0.22613065],
       [0.23115578],
       [0.236

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [13]:
from sklearn.linear_model import LinearRegression

In [20]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [21]:
lr = LinearRegression(num_iter = 5000000)
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

TypeError: LinearRegression.__init__() got an unexpected keyword argument 'num_iter'

In [19]:
lr.coef_

array([34.82170159,  0.39144139])

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [16]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [17]:
sum((clf.predict(X) - y)**2)

805.2567476112227

In [18]:
clf.coef

matrix([[-0.25254598],
        [ 2.11924863],
        [ 0.3622979 ]])